# Tutorial 4 - Hierarchical Agents

## Functionalities:
- An agent (referred as meta agent) can also assign other agents (referred as inner agents) as the function call, thereby creating a hierarchical structure
- Whenever the inner agent is called, we will run the full inner agent's process until termination
- Maximum number of steps for each agent (meta agent or inner agent) can be defined in `max_subtasks` (default: 5) variable when initialising agent

## Additional Details:
- The inner agent will take in meta agent's instruction as input
- In addition, the inner agent will also have access to meta agent's task and subtasks completed for global context
- Inner agent's output will directly go into meta agent's subtasks completed dictionary, enabling it to directly update global context

## Design Philosophy:
- Bottom layers can get context of the top layers for more context-specific generation
- Top layers choose bottom layers in a hierarchical fashion so as to limit the number of choices at each level for better performance
- `subtasks_completed` serves as the global context for the meta agents and the inner agents, and can be summarised as needed, e.g. when performing a new task to filter out irrelevant information 

# Setup Guide

## Step 1: Install TaskGen

In [1]:
# !pip install taskgen-ai

## Step 2: Import required functions and setup relevant API keys for your LLM

In [2]:
# Set up API key and do the necessary imports
from taskgen import *
import os

# this is only if you use OpenAI as your LLM
os.environ['OPENAI_API_KEY'] = '<YOUR API KEY HERE>'

## Step 3: Define your own LLM
- Take in a `system_prompt`, `user_prompt`, and outputs llm response string

In [3]:
def llm(system_prompt: str, user_prompt: str) -> str:
    ''' Here, we use OpenAI for illustration, you can change it to your own LLM '''
    # ensure your LLM imports are all within this function
    from openai import OpenAI
    
    # define your own LLM here
    client = OpenAI()
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        temperature = 0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content

In [4]:
# Verify that llm function is working
llm(system_prompt = 'You are a classifier to classify the sentiment of a sentence', 
    user_prompt = 'It is a hot and sunny day')

'Neutral'

# Define Agents and Meta Agents
- You can also create a Meta Agent that uses other Agents (referred to as Inner Agents) as functions
- Create your Meta agent using `Agent()` (Note: No different from usual process of creating Agents - your Meta Agent is also an Agent)
- Set up an Inner Agent list and assign it to your Meta agent using `assign_agents(agent_list)`

## Example Meta Agent Setup
```python
# Define your meta-agent
my_agent = Agent('Menu Creator', 
                 'Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.',
                 llm = llm)

# Define your agent list. Note you can just assign functions to the agent in place using .assign_functions(function_list)
agent_list = [
    Agent('Chef', 'Takes in dish names and generates ingredients for each of them. Does not generate prices.', llm = llm),
    Agent('Boss', 'Makes decision on which items to include in menu.', llm = llm),
    Agent('Creative Writer', 'Takes in a cuisine type and generates interesting dish names and descriptions.', max_subtasks = 2, llm = llm),
    Agent('Economist', 'Takes in dish names and comes up with pricing for each of them', llm = llm).assign_functions([fp])
    ]

my_agent.assign_agents(agent_list)
```

## Run the Meta Agent
- Let us run the agent and see the interactions between the Meta Agent and Inner Agents to solve the task!
```python
output = my_agent.run('Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Ensure all parts of menu are generated.')
```

In [5]:
def random_number_from_string(input_string) -> int:
    ''' Returns a random number between 1 and 10 when given an input string '''
    import hashlib
    import random
    # Hash the input string
    hashed_string = hashlib.sha256(input_string.encode()).hexdigest()
    
    # Convert the hashed string to an integer
    hashed_integer = int(hashed_string, 16)
    
    # Seed the random number generator with the hashed integer
    random.seed(hashed_integer)
    
    # Generate a random number between 1 and 10
    return random.randint(1, 10)

In [6]:
# assign a dish pricing function for economist
# Note: Try to make the output informative that is understandable by the Agent - use free text or json to explain the output so that someone can understand without the input
def dish_price(list_of_dish_names: list) -> dict:
    ''' Takes in list_of_dish_names and outputs price of each dish '''
    if not isinstance(list_of_dish_names, list):
        list_of_food_items = [list_of_dish_names]
    output = {}
    for each in list_of_dish_names:
        output[each] = '$'+str(random_number_from_string(each))
    return output

In [7]:
# Define your meta-agent
my_agent = Agent('Menu Creator', 
                 'Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.',
                 llm = llm)

In [8]:
# Define your agent list. Note you can just assign functions to the agent in place using .assign_functions(function_list)
agent_list = [
    Agent('Chef', 'Takes in dish names and generates ingredients for each of them. Does not generate prices.', llm = llm),
    Agent('Boss', 'Makes decisions on which items to include in Menu', llm = llm),
    Agent('Creative Writer', 'Takes in a cuisine type and generates interesting dish names and descriptions. Does not generate prices or ingredients.', 
          max_subtasks = 1, llm = llm),
    Agent('Economist', 'Takes in dish names and comes up with pricing for each of them', llm = llm).assign_functions([dish_price])
    ]

In [9]:
my_agent.assign_agents(agent_list)

# Approach 1: Run a generic task with the Meta-Agent
- Let us run a general task to create 5 menu items for an Italian restaurant
- You can also specify how Agents interact with one another in the task, otherwise the Meta-Agent will infer what should be done

In [10]:
output = my_agent.run('Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Output in Menu format.')

Observation: No subtasks have been completed yet for the assigned task of generating 5 menu items based on Italian food choices.
Thoughts: To complete the assigned task, I need to first generate interesting dish names and descriptions for Italian cuisine. After that, I will generate the ingredients for each dish and finally determine the pricing.
Subtask identified: Generate interesting dish names and descriptions for 5 Italian menu items.
Calling function Creative Writer with parameters {'instruction': 'Generate interesting dish names and descriptions for 5 Italian menu items.'}

### Start of Inner Agent: Creative Writer ###
Observation: No subtasks have been completed yet for generating interesting dish names and descriptions for Italian menu items.
Thoughts: To complete the assigned task, I need to generate five unique and creative dish names along with their descriptions that reflect Italian cuisine.
Subtask identified: Use the equipped function to generate interesting dish names a

In [11]:
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Shared Variables: ['agent']
Task: Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Output in Menu format.
Subtasks Completed:
Subtask: Creative Writer(instruction="Generate interesting dish names and descriptions for 5 Italian menu items.")
{'Agent Output': '1. **Trattoria Tiramisu Delight**: A luscious twist on the classic dessert, this dish features layers of espresso-soaked ladyfingers, rich mascarpone cream, and a hint of orange zest, all dusted with cocoa powder for a delightful finish.\n\n2. **Pasta al Pesto Paradiso**: A heavenly blend of al dente linguine tossed in a vibrant basil pesto, adorned with sun-dried tomatoes and toasted pine nuts, creating a symphony of flavors that transport you straight to 

In [12]:
output = my_agent.reply_user()

1. **Trattoria Tiramisu Delight**  
   - **Description**: A luscious twist on the classic dessert, this dish features layers of espresso-soaked ladyfingers, rich mascarpone cream, and a hint of orange zest, all dusted with cocoa powder for a delightful finish.  
   - **Ingredients**: ladyfingers, espresso, mascarpone cheese, sugar, eggs, orange zest, cocoa powder.  
   - **Pricing**: $2  

2. **Pasta al Pesto Paradiso**  
   - **Description**: A heavenly blend of al dente linguine tossed in a vibrant basil pesto, adorned with sun-dried tomatoes and toasted pine nuts, creating a symphony of flavors that transport you straight to the Italian Riviera.  
   - **Ingredients**: linguine, fresh basil, garlic, pine nuts, olive oil, sun-dried tomatoes, Parmesan cheese, salt, and pepper.  
   - **Pricing**: $1  

3. **Sicilian Citrus Caprese**  
   - **Description**: A refreshing take on the traditional Caprese salad, this dish combines ripe heirloom tomatoes, creamy burrata, and a medley of cit

# Approach 2: Step through the process yourself as the Meta Agent
- You can use agents manually using `use_agent(agent_name: str, agent_task: str)` The outputs of the agents will be stored into `subtasks_completed` of the meta agent automatically
- You can also interact with the meta agent using `reply_user(task: str, stateful: bool = False)`. This generates a reply for the agent to the `task` using `subtasks_completed` as context. If `stateful` is true, this will also store the output into `subtasks_completed`
- If `subtasks_completed` is too lengthy, you can run `summarise_subtasks_completed(task: str)` This summarises `subtasks_completed` according to what is relevant to the `task`
- If you want to redo a subtask, you can remove the most recent subtask from `subtasks_completed` using `remove_last_subtask()`

In [13]:
# reset the state of the agent
my_agent.reset()
# currently should have no task and nothing in subtasks_completed
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Shared Variables: ['agent']
Task: No task assigned
Subtasks Completed: None
Is Task Completed: False


In [14]:
# Assign a task for the meta agent - this helps to contextualise what you give each of the inner agents later on
my_agent.assign_task('Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Output in Menu format.')

In [15]:
# This will run the Agent Creative Writer (with shared subtasks_completed as meta agent)
# Output of task will go directly into subtasks_completed - we view the inner agent as simply an extension of the meta agent
my_agent.use_agent('Creative Writer', 'Generate only 5 Italian dish names and descriptions')

Calling function Creative Writer with parameters {'instruction': 'Generate only 5 Italian dish names and descriptions'}

### Start of Inner Agent: Creative Writer ###
Observation: No subtasks have been completed yet for generating Italian dish names and descriptions.
Thoughts: To complete the assigned task, I need to generate 5 unique Italian dish names along with their descriptions. I can utilize the general function to create these names and descriptions.
Subtask identified: Use the equipped function to generate 5 interesting Italian dish names and their descriptions.
Getting LLM to perform the following task: Use the equipped function to generate 5 interesting Italian dish names and their descriptions.
> 1. **Sogni di Pomodoro**: A dreamy blend of sun-ripened tomatoes and fragrant basil, this dish features a velvety tomato sauce draped over al dente pasta, evoking the warmth of an Italian summer. 

2. **Frittata di Mare**: Dive into the ocean with this exquisite seafood frittata, wh

In [16]:
# subtasks_completed should be updated with the Creative Agent's outputs
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Shared Variables: ['agent']
Task: Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Output in Menu format.
Subtasks Completed:
Subtask: Creative Writer(instruction="Generate only 5 Italian dish names and descriptions")
{'Agent Output': '1. **Sogni di Pomodoro**: A dreamy blend of sun-ripened tomatoes and fragrant basil, this dish features a velvety tomato sauce draped over al dente pasta, evoking the warmth of an Italian summer. \n\n2. **Frittata di Mare**: Dive into the ocean with this exquisite seafood frittata, where tender shrimp and succulent calamari are nestled in a fluffy egg base, seasoned with fresh herbs and a hint of lemon zest. \n\n3. **Risotto dell’Amore**: A creamy risotto infused with the essence

In [17]:
my_agent.use_agent('Chef', 'Generate ingredients for the 5 Italian dishes')

Calling function Chef with parameters {'instruction': 'Generate ingredients for the 5 Italian dishes'}

### Start of Inner Agent: Chef ###
Observation: The task of generating 5 Italian dish names and descriptions has been completed successfully, providing a variety of dishes that reflect Italian cuisine.
Thoughts: Now, I need to generate the ingredients for each of the 5 Italian dishes that have been described. This will involve creating a list of ingredients that correspond to each dish, ensuring they are appropriate and authentic to the recipes.
Subtask identified: Generate a list of ingredients for each of the following dishes: Sogni di Pomodoro, Frittata di Mare, Risotto dell’Amore, Pollo al Limone e Rosmarino, and Torta di Ricotta e Miele.
Getting LLM to perform the following task: Generate a list of ingredients for each of the following dishes: Sogni di Pomodoro, Frittata di Mare, Risotto dell’Amore, Pollo al Limone e Rosmarino, and Torta di Ricotta e Miele.
> 1. **Sogni di Pomod

In [18]:
my_agent.use_agent('Economist', 'Generate prices for the 5 Italian dishes')

Calling function Economist with parameters {'instruction': 'Generate prices for the 5 Italian dishes'}

### Start of Inner Agent: Economist ###
Observation: The task of generating prices for the 5 Italian dishes has not yet been started, but the dish names and descriptions have been successfully created.
Thoughts: To complete the assigned task, I need to utilize the dish names generated earlier to determine appropriate pricing for each dish. This can be done by passing the list of dish names to the dish_price function.
Subtask identified: Use the dish_price function to input the list of the 5 Italian dish names and receive the corresponding prices for each dish.
Calling function dish_price with parameters {'list_of_dish_names': ['Sogni di Pomodoro', 'Frittata di Mare', 'Risotto dell’Amore', 'Pollo al Limone e Rosmarino', 'Torta di Ricotta e Miele']}
> {'output_1': {'Sogni di Pomodoro': '$9', 'Frittata di Mare': '$2', 'Risotto dell’Amore': '$8', 'Pollo al Limone e Rosmarino': '$10', 'To

In [19]:
my_agent.use_agent('Boss', 'Choose only 3 Italian dishes to serve that will please Gordon Ramsay')

Calling function Boss with parameters {'instruction': 'Choose only 3 Italian dishes to serve that will please Gordon Ramsay'}

### Start of Inner Agent: Boss ###
Observation: I have generated five Italian dishes along with their descriptions, ingredients, and prices, but no selection has been made yet for the three dishes to serve.
Thoughts: To choose three Italian dishes that will please Gordon Ramsay, I should consider the complexity, presentation, and flavor profiles of the dishes. I can analyze the descriptions and ingredients to select the most appealing options.
Subtask identified: Evaluate the five Italian dishes based on their descriptions and ingredients to select the top three that would impress Gordon Ramsay, focusing on quality and culinary creativity.
Getting LLM to perform the following task: Evaluate the five Italian dishes based on their descriptions and ingredients to select the top three that would impress Gordon Ramsay, focusing on quality and culinary creativity.
> 

In [20]:
# Get a targeted response from your agent, and add this to subtask_completed
output = my_agent.reply_user('Choose the 3 menu items for Gordon Ramsay line by line in this format - Dish Name, Dish Description, Dish Ingredients, Dish Price')

1. **Risotto dell’Amore**: A creamy risotto infused with the essence of love, this dish combines Arborio rice with rich saffron and a medley of seasonal vegetables, creating a comforting and romantic experience. 
   **Ingredients**: Arborio rice, saffron threads, vegetable or chicken broth, seasonal vegetables (such as peas and asparagus), onion, garlic, Parmesan cheese, salt, and pepper. 
   **Price**: $8 

2. **Pollo al Limone e Rosmarino**: Experience the zesty delight of lemon and rosemary marinated chicken, grilled to perfection and served with a side of roasted potatoes, capturing the essence of Italian countryside cooking. 
   **Ingredients**: Chicken breasts, fresh lemon juice, rosemary, garlic, olive oil, salt, pepper, and potatoes for roasting. 
   **Price**: $10 

3. **Torta di Ricotta e Miele**: Indulge in a sweet symphony with this ricotta and honey tart, featuring a buttery crust and a luscious filling, finished with a drizzle of local honey and a sprinkle of crushed pist

In [21]:
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Shared Variables: ['agent']
Task: Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Output in Menu format.
Subtasks Completed:
Subtask: Creative Writer(instruction="Generate only 5 Italian dish names and descriptions")
{'Agent Output': '1. **Sogni di Pomodoro**: A dreamy blend of sun-ripened tomatoes and fragrant basil, this dish features a velvety tomato sauce draped over al dente pasta, evoking the warmth of an Italian summer. \n\n2. **Frittata di Mare**: Dive into the ocean with this exquisite seafood frittata, where tender shrimp and succulent calamari are nestled in a fluffy egg base, seasoned with fresh herbs and a hint of lemon zest. \n\n3. **Risotto dell’Amore**: A creamy risotto infused with the essence

In [22]:
# you can summarise the subtasks_history for the next task
my_agent.summarise_subtasks_completed('Generate the 3 Italian dishes for Gordon Ramsay in menu format')

1. **Risotto dell’Amore**: A creamy risotto infused with the essence of love, this dish combines Arborio rice with rich saffron and a medley of seasonal vegetables, creating a comforting and romantic experience.  
   **Ingredients**: Arborio rice, saffron threads, vegetable or chicken broth, seasonal vegetables (such as peas and asparagus), onion, garlic, Parmesan cheese, salt, and pepper.  
   **Price**: $8  

2. **Pollo al Limone e Rosmarino**: Experience the zesty delight of lemon and rosemary marinated chicken, grilled to perfection and served with a side of roasted potatoes, capturing the essence of Italian countryside cooking.  
   **Ingredients**: Chicken breasts, fresh lemon juice, rosemary, garlic, olive oil, salt, pepper, and potatoes for roasting.  
   **Price**: $10  

3. **Torta di Ricotta e Miele**: Indulge in a sweet symphony with this ricotta and honey tart, featuring a buttery crust and a luscious filling, finished with a drizzle of local honey and a sprinkle of crushe

In [23]:
# subtask has been summarised, helping to reduce the context significantly for later task generation
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Shared Variables: ['agent']
Task: Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Output in Menu format.
Subtasks Completed:
Subtask: Current Results for 'Generate the 3 Italian dishes for Gordon Ramsay in menu format'
1. **Risotto dell’Amore**: A creamy risotto infused with the essence of love, this dish combines Arborio rice with rich saffron and a medley of seasonal vegetables, creating a comforting and romantic experience.  
   **Ingredients**: Arborio rice, saffron threads, vegetable or chicken broth, seasonal vegetables (such as peas and asparagus), onion, garlic, Parmesan cheese, salt, and pepper.  
   **Price**: $8  

2. **Pollo al Limone e Rosmarino**: Experience the zesty delight of lemon and rosemar

In [24]:
# we also allow you to manually remove the last item added to subtasks_completed, so you can revert your agent state if needed for backtracking
my_agent.remove_last_subtask()

Removed last subtask from subtasks_completed: ("Current Results for 'Generate the 3 Italian dishes for Gordon Ramsay in menu format'", '1. **Risotto dell’Amore**: A creamy risotto infused with the essence of love, this dish combines Arborio rice with rich saffron and a medley of seasonal vegetables, creating a comforting and romantic experience.  \n   **Ingredients**: Arborio rice, saffron threads, vegetable or chicken broth, seasonal vegetables (such as peas and asparagus), onion, garlic, Parmesan cheese, salt, and pepper.  \n   **Price**: $8  \n\n2. **Pollo al Limone e Rosmarino**: Experience the zesty delight of lemon and rosemary marinated chicken, grilled to perfection and served with a side of roasted potatoes, capturing the essence of Italian countryside cooking.  \n   **Ingredients**: Chicken breasts, fresh lemon juice, rosemary, garlic, olive oil, salt, pepper, and potatoes for roasting.  \n   **Price**: $10  \n\n3. **Torta di Ricotta e Miele**: Indulge in a sweet symphony wit

In [25]:
# visualise your agent's status - should have empty subtasks_completed
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Shared Variables: ['agent']
Task: Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Output in Menu format.
Subtasks Completed: None
Is Task Completed: False
